<a href="https://colab.research.google.com/github/danilotpnta/UN-General-Debate-Analysis-SDGs/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UN-General-Debate-Analysis-SDGs

This project analyzes the UN General Debate Corpus from 1970 to 2023. It includes exploratory data analysis (EDA), predictive modeling, and data visualizations focusing on uncovering insights from political speeches and their connection to global challenges.

The project is divided into the following sections:

1. Data Preprocessing
2. Exploratory Data Analysis
3. Predictive Modeling
4. Data Visualization

## Data Preprocessing

#### Donwloading the dataset

In [ ]:
from utils.dataverse_downloader import download_dataset

# Call the function to download all files
download_dataset()

File already downloaded: Raw_PDFs_1946-1969.tgz
File already downloaded: Raw_PDFs_1970-1990.tgz
File already downloaded: Raw_PDFs_1991-2022.tgz
File already downloaded: README.txt
File already downloaded: Speakers_by_session.xlsx
File already downloaded: UNGDC_1946-2023.tgz


## Exploratory Data Analysis


Testing updating github repo